# Seasonal Decompose

An example of how to decompose time series into trend, seasonal, and residual components.
It uses the function within the ProcessData class from the COAsT package that will decompose time series into trend, seasonal and residual components. The function is a wrapper that adds functionality to the `seasonal_decompose` function contained in the [statsmodels](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html#statsmodels.tsa.seasonal.seasonal_decompose) package to make it more convenient for large geospatial datasets.

Specifically:
1) Multiple time series spread across multiple dimensions, e.g. a gridded dataset, can be processed. The user simply passes in an xarray DataArray that has a "t_dim" dimension and 1 or more additional dimensions, for example gridded spatial dimensions
2) Masked locations, such as land points, are handled
3) The decomposed time series are returned as xarray DataArrays within a single coast.Gridded object

## An example
Below is an example using the `coast.ProcessData.seasonal_decomposition` function with the example data. Note that we will artifically extend the length of the example data time series for demonstrative purposes.

Begin by importing coast, defining paths to the data, and loading the example data into a gridded object:

In [4]:
import warnings 
warnings.filterwarnings('ignore')

In [5]:
import coast
import numpy as np
import xarray as xr
import glob

In [6]:
root = "../../"
dn_files = root + "./example_files/"

fn_nemo_dat  = dn_files + "coast_example_nemo_data.nc"
# Set path for domain file if required.
fn_nemo_dom  = dn_files + "coast_example_nemo_domain.nc"
# Set path for model configuration file
config = root + "./config/example_nemo_grid_t.json"

# Read in data (This example uses NEMO data.)
grd1 = coast.Gridded(fn_nemo_dat, fn_nemo_dom, config=config)

FileNotFoundError: [Errno 2] No such file or directory: '/home/users/colinsau/example_files/coast_example_nemo_data.nc'

In [ ]:
data_path = "/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/1"
fn_nemo_dom = "/gws/nopw/j04/canari/users/dlrhodso/mesh_mask.nc"
config_t = "../config/example_nemo_grid_t.json"
years = ['2010', '2011', '2012', '2013']

data_files = []
for year in years:
    infiles = glob.glob((f"{data_path}/OCN/yearly/{year}/*_votemper.nc"))
    for infile in infiles:
        data_files.append(infile)

In [ ]:
fn_nemo_dat = xr.open_mfdataset(data_files)
grd = coast.Gridded(fn_data=fn_nemo_dat,
                          fn_domain=fn_nemo_dom,
                          config=config_t,
                          )

The loaded example data only has 7 time stamps, the code below creates a new (fake) extended temperature variable with 48 monthly records. This code is not required to use the function, it is only included here to make a set of time series that are long enough to be interesting.

Subsetting data by depth

In [ ]:
grd = grd.isel(z_dim=[0,1])

In [ ]:
temperature = grd.dataset.temperature

In [ ]:
temperature

Check out the new data

In [ ]:
#temperature # uncomment to print data object summary

In [ ]:
temperature[0,0,:,:].plot()

Check out time series at 2 different grid points

In [ ]:
temperature[:,0,800,800].plot()
temperature[:,0,1000,1000].plot()

Create a `coast.Process_data` object, and call the `seasonal_decomposition` function, passing in the required arguments. The first two arguments are:
1. The input data, here the temperature data as an xarray DataArray
2. The number of chuncks to split the data into. Here we split the data into 2 chunks so that the dask scheduler will try to run 4 processes in parallel

The remaining arguments are keyword arguments for the underlying `statsmodels.tsa.seasonal.seasonal_decompose` function, which are documented on the [statsmodels documentation pages](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html#statsmodels.tsa.seasonal.seasonal_decompose).  Here we specify:

3. three The type of model, i.e. an additive model
4. The period of the seasonal cycle, here it is 6 months
5. Extrapolate the trend component to cover the entire range of the time series (this is required because the trend is calculated using a convolution filter) 


In [ ]:
proc_data = coast.ProcessData()
grd = proc_data.seasonal_decomposition(temperature, 2, model="additive", period=6, extrapolate_trend="freq")

The returned xarray Dataset contains the decomposed time series (trend, seasonal, residual) as dask arrays

In [ ]:
grd.dataset # uncomment to print data object summary

Execute the computation

In [ ]:
grd.dataset.compute()

Plot the decomposed time series

In [ ]:
component = xr.DataArray( ["trend","seasonal","residual"], dims="component", name="component" )
temp_decomp = xr.concat( 
    [grd.dataset.trend, grd.dataset.seasonal,grd.dataset.residual], dim=component 
)
temp_decomp.name = "temperature"
temp_decomp[:,:,0,800,800].plot(hue="component")